In [1]:
import torch
from multimodal_particles.models import AbsorbingFlow
from multimodal_particles.config_classes.absorbing_flows_config import AbsorbingConfig
from multimodal_particles.data.particle_clouds.jets import JetDataclass
from multimodal_particles.data.particle_clouds.jets_dataloader import JetsDataloaderModule
from multimodal_particles.models.generative.bridges import AbsorbingBridge

In [2]:
# Define parameters
config = AbsorbingConfig()
t = torch.tensor([0.5,0.4, 1.0], dtype=torch.float32)  # Example input (B,)

config = AbsorbingConfig()
model = AbsorbingFlow(config)
random_databatch = JetsDataloaderModule.random_databatch(config)

/home/cesarali/particles/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [50]:
t = torch.ones(
    random_databatch.target_continuous.shape[0], device=random_databatch.target_continuous.device
).type_as(random_databatch.target_continuous)

# ts = self.min_t + (1-self.min_t) * torch.rand((B,)) # (B,)

time = model.reshape_time(t, random_databatch.target_continuous)

B = random_databatch.target_continuous.size(0)
N = random_databatch.target_continuous.size(1)
mask_1 = random_databatch.target_mask
time_ = time.repeat((1,N,1)) # (B,N,1)
random_ = torch.rand_like(time_)
survival_probability = model.bridge_absorbing.survival_probability(time_)
mask_t = (random_ < survival_probability ).long()
where_alive = torch.where(mask_1)
mask_t[where_alive] = 1

In [54]:
(mask_t == mask_1).all()

tensor(True)